In [1]:
# ============================================================
# CONFIGURACIÓN GLOBAL (Reproducibilidad y modo rápido)
# ============================================================
# Semilla para que el split y los resultados sean repetibles
RANDOM_SEED = 42

# Tamaño del set de prueba
TEST_SIZE = 0.2

# Definición de "Best Seller" como TOP 25% en quantity_sold (percentil 75)
THRESH_Q = 0.75

# Submuestra por clase (0 y 1). Esto hace el notebook rápido en cualquier laptop.
# (Si tu PC aguanta más, sube a 2000 o 3000)
N_POR_CLASE = 1000

# Iteraciones ADVI (inferencia variacional) y cantidad de muestras del "trace"
# Mantener bajo para demo rápida; subir si tienes compilador y más tiempo.
ADVI_ITERS = 1000
TRACE_SAMPLES = 1000

# 1 — Contexto y objetivo

## Modelo 2 — Clasificador de Éxito (Regresión Logística Bayesiana)

**Objetivo:** estimar la probabilidad de que un producto sea un *Best Seller* (venta alta) usando como predictores `rating` y `discounted_price`.

### ¿Qué es “Best Seller”?

Lo definimos como estar en el **top 25% de `quantity_sold`**.  
Esto convierte el problema en una **clasificación binaria**:

- **1 = Best Seller**  
- **0 = No Best Seller**

### ¿Por qué un modelo Bayesiano?

Porque no solo queremos una predicción puntual (0/1), sino también **incertidumbre**:  
una **distribución posterior de probabilidades** y un **intervalo creíble**.

### Nota de ingeniería

El muestreo MCMC completo puede ser lento en notebooks (especialmente sin compilación C++).  
Para mantener el flujo rápido y reproducible, utilizamos:

- **Submuestra estratificada** (balanceada por clase)  
- **ADVI (Variational Inference)** para aproximar la posterior de forma eficiente  

In [ ]:
import polars as pl
import numpy as np

# ============================================================
# 1) CARGA Y LIMPIEZA DE DATOS
# ============================================================
# Leemos el CSV con Polars (más eficiente que pandas para ETL)
df = pl.read_csv("../dataset/amazon_sales_dataset.csv")

# Seleccionamos SOLO columnas necesarias para el Modelo 2:
# - rating: predictor
# - discounted_price: predictor
# - quantity_sold: lo usamos para construir el target "best_seller"
df2 = (
    df.select(["rating", "discounted_price", "quantity_sold"])
      .drop_nulls()
      .filter(pl.col("rating").is_between(0, 5))     # ratings válidos
      .filter(pl.col("discounted_price") > 0)        # precios válidos
      .filter(pl.col("quantity_sold") >= 0)          # ventas válidas
)

print("Filas después de limpieza:", df2.height)
df2.head()

FileNotFoundError: El sistema no puede encontrar la ruta especificada. (os error 3): dataset/amazon_sales_dataset.csv